In [21]:
#importing libraries
import os
import numpy as np
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPool2D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

In [10]:
# get all the data in the directory train and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'data/seg_train/', 
        target_size=(150, 150), batch_size=14034)

# get all the data in the directory test and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'data/seg_test/', 
        target_size=(150, 150), batch_size=3000) 


# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [11]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]


print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))

print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))


Number of training samples: 14034
Number of testing samples: 3000
train_images shape: (14034, 150, 150, 3)
train_labels shape: (14034, 6)
test_images shape: (3000, 150, 150, 3)
test_labels shape: (3000, 6)


In [12]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)


(14034, 67500)
(3000, 67500)


In [13]:
train_y = np.reshape(train_labels[:,0], (14034,1))
test_y = np.reshape(test_labels[:,0], (3000,1))

In [14]:
# Build a baseline fully connected model
from keras import models
from keras import layers
np.random.seed(42)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(67500,)))
 
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='relu'))

model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

In [15]:
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                     train_y,
                    epochs=5,
                    batch_size=32)

Epoch 1/5
439/439 [==============================] - 37s 84ms/step - loss: 1.8611e-08 - accuracy: 0.6220
Epoch 2/5
439/439 [==============================] - 16s 36ms/step - loss: 1.8611e-08 - accuracy: 0.6227
Epoch 3/5
439/439 [==============================] - 13s 29ms/step - loss: 1.8611e-08 - accuracy: 0.6230
Epoch 4/5
439/439 [==============================] - 12s 27ms/step - loss: 1.8611e-08 - accuracy: 0.62311s
Epoch 5/5
439/439 [==============================] - 12s 26ms/step - loss: 1.8611e-08 - accuracy: 0.6259


In [17]:
results_train = model.evaluate(train_img, train_y)

439/439 [==============================] - 23s 53ms/step - loss: 1.8611e-08 - accuracy: 0.6366


In [18]:
results_train

[1.8611055807582488e-08, 0.6365968585014343]

In [25]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                     train_y,
                    epochs=5,
                    batch_size=32)

Epoch 1/5


ValueError: in user code:

    c:\Users\Wendy\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\Users\Wendy\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\Wendy\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\Wendy\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\Wendy\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\Wendy\anaconda3\envs